In [2]:
from datasets import load_dataset

dataset = load_dataset("kevykibbz/Amazon_Customer_Review_2023")

ModuleNotFoundError: No module named 'secrets'

## Loading a pretrained model and its tokenizer and using for a particular tasks

In [6]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [9]:
tokenizer =  AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")
model = TFAutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [10]:
# sample sequence classification tasks
data = [
    'I love the game of football',
    'I have missed feelings with swimming games'
]

In [1]:
tokenizer?

Object `tokenizer` not found.


In [12]:
# preprocess input with tokenizer
input = tokenizer(data, truncation=True, padding=True, return_tensors='tf' )
print(input)

{'input_ids': <tf.Tensor: shape=(2, 9), dtype=int32, numpy=
array([[ 101, 1045, 2293, 1996, 2208, 1997, 2374,  102,    0],
       [ 101, 1045, 2031, 4771, 5346, 2007, 5742, 2399,  102]],
      dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 9), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}


In [13]:
output = model(input)
print(output)

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-4.2080727,  4.5056076],
       [ 2.2848012, -1.9209454]], dtype=float32)>, hidden_states=None, attentions=None)


In [14]:
output.logits

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-4.2080727,  4.5056076],
       [ 2.2848012, -1.9209454]], dtype=float32)>

In [16]:
import tensorflow as tf
prediction = tf.math.softmax(output.logits)
print(prediction)

tf.Tensor(
[[1.6429540e-04 9.9983573e-01]
 [9.8530942e-01 1.4690619e-02]], shape=(2, 2), dtype=float32)


In [24]:
# uisng tokenize method and convert string to id methods of the tokenizer to achieve similar result
tokens = tokenizer.tokenize(data)
print(tokens)

['i', 'love', 'the', 'game', 'of', 'football', 'i', 'have', 'missed', 'feelings', 'with', 'swimming', 'games']


In [25]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

[1045, 2293, 1996, 2208, 1997, 2374, 1045, 2031, 4771, 5346, 2007, 5742, 2399]


## Fine-tuning a model for a specific tasks

In [6]:
from datasets import load_dataset

ModuleNotFoundError: No module named 'secrets'

In [ ]:
# loading dataset from HF hub
raw_datasets = load_dataset('glue', 'mrpc')
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [29]:
raw_datasets['train'][0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [ ]:
raw_datasets_train =  raw_datasets['train']
raw_datasets_train.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [ ]:
# inspecting a two records from our train dataset
raw_datasets['train'][:2]

{'sentence1': ['Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
  "Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion ."],
 'sentence2': ['Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
  "Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 ."],
 'label': [1, 0],
 'idx': [0, 1]}

In [36]:
tokens = tokenizer('I love the game of football', 'I have missed feelings with swimming games', 
                   padding=True, 
                   truncation=True, 
                   return_tensors='tf',
                   return_token_type_ids = True
                   )
tokens

{'input_ids': <tf.Tensor: shape=(1, 16), dtype=int32, numpy=
array([[ 101, 1045, 2293, 1996, 2208, 1997, 2374,  102, 1045, 2031, 4771,
        5346, 2007, 5742, 2399,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 16), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 16), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [ ]:
# an efficient and fast way to tokenize all input sequences would be put to create a function and then apply the function using 
# the map function to the datasets

def tokenize_function(data):
    #NB:
    # 1. a tokenizer can take in separate sequences 
    # 2. will not apply batching here, because memory efficiency issues
    return tokenizer(data['sentence1'], data['sentence2'], truncation=True)

In [38]:
tokenized_dataset = raw_datasets.map(tokenize_function, batched=True)
tokenized_dataset

Map: 100%|██████████| 1725/1725 [00:00<00:00, 22973.99 examples/s]


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [ ]:
# using dynamic padding to pad our input sequnencs as another option instead of the default padding parameter in the tokenizer function
# so we can now pad our tokenized dataset


